In [28]:
import pandas as pd
import pickle

from os import listdir
from os.path import isfile, join

In [29]:
folder_name = '2018-04-20 02-38-11'

In [30]:

def trimming(file_name):
    # 1. 역순으로 제일 먼저 찾은 이메일 밑으로 다 없앰
    # 2. 나머지 특수문자들 다 없앰 (. 빼고)
    # 3. 공백문자열 다 없앰
    import re
    with open(file_name,'r',encoding='utf8') as f:
        articles = f.readlines()
    
    # 1.
    is_email = []
    for each in articles:
        tmp = re.findall('[^@]+@[^@]+\.[^@]+', each)
        is_email.append(tmp)
    
    is_email = [True if len(each)!=0 else False for each in is_email]
    upper_bound = len(is_email) - 2 - is_email[::-1].index(True) if True in is_email else len(is_email)-1
    
    articles = [articles[idx] if not is_email[idx] else '' for idx in range(0,upper_bound)]
    
    # 2.
    for idx, val in enumerate(articles):
        converted = re.sub('[^가-힣0-9a-zA-Z.\\s]', ' ', val)
        articles[idx] = converted
    
    # 3.
    articles = [each for each in articles if each != '']
    
    return articles

In [31]:
# file_name = '정치/0-2018-04-14 17-03.txt'
# trimmed = trimming(file_name)
# trimmed = ''.join(trimmed)

In [32]:
# with open(file_name,'w',encoding='utf-8') as f:
#     f.write(trimmed)

In [33]:
# from os import listdir
# from os.path import isfile, join
# path = '정치'

# file_names = [f for f in listdir(path) if isfile(join(path, f)) and  f.endswith(".txt")]
# full_names = ['{}/{}'.format(path, each) for each in file_names ]

In [34]:
from datetime import datetime

now = datetime.now()
cur_time = now.strftime('%Y-%m-%d %H-%M-%S')

In [35]:
from os import makedirs as mk
from os import path as pt

path = ['정치','경제','사회','세계','생활문화','IT과학']

new_path = '{}/{}'.format('preprocess',folder_name)

if not pt.exists(new_path):
    mk(new_path)
    for area in path:
        mk('{}/{}'.format(new_path,area))

In [36]:
from os import listdir
from os.path import isfile, join

for each in path:
    crawling_path = '{}/{}/{}'.format('crawling',folder_name,each)
    file_names = [f for f in listdir(crawling_path) if isfile(
        join(crawling_path, f)) and f.endswith(".txt")]
    # 파일이름

    full_names = ['{}/{}'.format(each, tmp) for tmp in file_names]
    # 영역/파일이름

    crawling_path = ['{}/{}/{}'.format('crawling',folder_name, tmp) for tmp in full_names]
    # crawling/영역/파일이름
    for idx, file_name in enumerate(crawling_path):
        trimmed = trimming(file_name)
        trimmed = ''.join(trimmed)
        
        dest_path = '{}/{}/{}'.format('preprocess',folder_name,full_names[idx])

        with open(dest_path, 'w', encoding='utf-8') as f:
            f.write(trimmed)

    print(each, ' completed !')

정치  completed !
경제  completed !
사회  completed !
세계  completed !
생활문화  completed !
IT과학  completed !


In [37]:
path = [('정치',0), ('경제',1), ('사회',2), ('세계',3), ('생활문화',4), ('IT과학',5)]
# path = [('정치',0)]

total_df = pd.DataFrame(columns=['content','area'])

def date_trimmer(raw_date):
    tmp = raw_date.split('/')[-1].replace('.txt',':00')
    tmp = tmp.split()
    return '-'.join(tmp[0].split('-')[1:]) + ' ' + tmp[1].replace('-',':')

for each in path:
    preprocess_path = '{}/{}/{}'.format('preprocess',folder_name,each[0])
    file_names = [f for f in listdir(preprocess_path) if isfile(
        join(preprocess_path, f)) and f.endswith(".txt")]
    full_names = ['{}/{}'.format(preprocess_path, tmp) for tmp in file_names]

    contents = []
    written_date = []
    
    df = pd.DataFrame(columns=['content','area'])
    
    for file_name in full_names:

        with open(file_name, 'r', encoding='utf-8') as f:
            contents.append(f.read())
            written_date.append(file_name)
            
    df['content'] = pd.Series(contents)
    df['area'] = each[1]
    df['written_date'] = pd.Series(written_date)
    df['written_date'] = df['written_date'].apply(date_trimmer)
    df['title'] = df['content'].apply(lambda x: x.split('\n')[0].replace('\t',''))
    
    df = df[['title','content','written_date','area']]
    
    total_df = total_df.append(df)
    print(each,' completed !')

total_df['area'] = total_df.area.astype('int64')
total_df.reset_index(drop=True, inplace=True)

('정치', 0)  completed !
('경제', 1)  completed !
('사회', 2)  completed !
('세계', 3)  completed !
('생활문화', 4)  completed !
('IT과학', 5)  completed !
